In [1]:
import pandera
import fugue
import pandas as pd
import numpy as np

In [2]:
a = np.random.randint(1, 100, 1000)
b = np.random.randint(1, 100, 1000)
test1 = pd.DataFrame({'a':a, 'b':b})
test2 = test1 + 100
test1['partition'] = 'a'
test2['partition'] = 'b'
test = test1.append(test2)

In [3]:
import pandera as pa
from pandera import Column, Check, DataFrameSchema

schema_test1 = pa.DataFrameSchema({
    "a": Column(pa.Int, Check.less_than(100)),
})

schema_test2 = pa.DataFrameSchema({
    "a": Column(pa.Int, Check.greater_than(99))
})

partition_schema = {"a": schema_test1, "b": schema_test2}

In [4]:
# schema: *
def validator(df:pd.DataFrame) -> pd.DataFrame:
    partition = df['partition'].iloc[0]
    schema = partition_schema[partition]
    schema.validate(df)
    return df

In [5]:
from fugue import FugueWorkflow
from fugue_spark import SparkExecutionEngine

with FugueWorkflow(SparkExecutionEngine) as dag:
    df = dag.df(test)
    df = df.partition(by=["partition"]).transform(validator)
    df.show(5)

SparkDataFrame
a:long|b:long|partition:str
------+------+-------------
154   |118   |b            
178   |118   |b            
146   |164   |b            
120   |137   |b            
160   |125   |b            
